# Python SCPI interface for controlling the OpenDrop over a serial port

This notebook demonstrates a simple Python-based serial interface for controlling the [OpenDrop](https://www.gaudi.ch/OpenDrop) over a serial port. The following links give some background on the Standard Commands for Programmable Instruments (SCPI): a syntax/command standard commonly used for programmable test and measurement equipment.

* [Wikipedia page](https://en.wikipedia.org/wiki/Standard_Commands_for_Programmable_Instruments)
* https://goughlui.com/2021/03/28/tutorial-introduction-to-scpi-automation-of-test-equipment-with-pyvisa/

This implementation makes use of the [Vrekrer scpi parser library](https://github.com/Vrekrer/Vrekrer_scpi_parser) for Arduino. Using SCPI should theoretically provide compatibility with [LabVIEW](https://en.wikipedia.org/wiki/LabVIEW) and/or [PyVISA](https://pyvisa.readthedocs.io/en/latest/), and provides a simple, text-based protocol that makes it easy to add other language bindings.

Note that this notebook requires a [custom firmware](https://github.com/sci-bots/OpenDrop/commit/769bed2d3c367adf2ff1a4c2b7cdbfc0f70d5d52) for the OpenDrop v3.2.

In [1]:
import serial
import numpy as np
import struct


class SerialProxy():
    def __init__(self, port):
        '''
        Initialize a SerialProxy object.

        Parameters
        ----------
        port : string
            Serial port name (e.g., 'COM1' or '/dev/ttyUSB0')
        '''
        self._serial = serial.Serial(port, 115200)
        
        (self.manufacturer, self.model, self.serial_number,
             self.software_version) = self.identify().split(',')
        
    def __del__(self):
        # Release the serial port
        self._serial.close()

    def identify(self):
        # Return a string that uniquely identifies the OpenDrop.
        # The string is of the form "GaudiLabs,<model>,<serial number>,<software revision>" .
        self._serial.write(b"*IDN?\n")
        return self._serial.readline().strip().decode()
    
    @property
    def voltage(self):
        '''
        Get the voltage.

        Returns
        ----------
        value : float
            RMS voltage.
        '''
        self._serial.write(b"VOLTAGE?\n")
        return float(self._serial.readline().strip())
    
    @voltage.setter
    def voltage(self, value):
        '''
        Set the voltage.

        Parameters
        ----------
        value : float
            RMS voltage.
        '''
        self._serial.write(b"VOLTAGE %f\n" % value)
    
    def set_state_of_channels(self, state):
        '''
        Set state of channels on device using state bytes.

        See also: `state_of_channels` (get)

        Parameters
        ----------
        states : list or np.array
            0 or 1 for each channel (size must be equal to the total
            number of channels).
        '''
        assert(len(state) == len(self.state_of_channels))
        
        # Cast the incoming state variable as a numpy byte array
        state_of_channels = np.array(state).astype(np.uint8)

        # Pack the channel array into a hex formatted string
        proxy._serial.write(b'CHAN:STAT ' + 
                            ''.join(['%02x' % x for x in np.packbits(
                                state_of_channels)]).encode() +
                            b'\n')
                            
    @property
    def state_of_channels(self):
        '''
        Get the state of the channels on device.

        See also: `set_state_of_channels`

        Returns
        ----------
        states : np.array
            0 or 1 for each channel.
        '''
        proxy._serial.write(b'CHAN:STAT?\n')
        state_str = proxy._serial.readline().strip().decode()
        
        # Unpack the channel state array from a hex formatted string into an
        # array.
        return np.unpackbits(np.array(
            struct.unpack('B'*16,bytearray.fromhex(state_str)), np.uint8))


# Delete any existing proxy object to prevent errors from trying to re-open
# the serial port
try:
    del(proxy)
except:
    pass


# Create a serial proxy object (change the port depending on your device)
proxy = SerialProxy('COM32')

In [2]:
# Turn on channel 8
state_of_channels = np.zeros(128)
state_of_channels[8] = 1
proxy.set_state_of_channels(state_of_channels)

In [3]:
# Turn on channels 10, 20 and 30
state_of_channels = np.zeros(128)
state_of_channels[[10, 20, 30]] = 1
proxy.set_state_of_channels(state_of_channels)

In [4]:
# Sweep across all channels
for i in range(128):
    state_of_channels = np.zeros(128)
    state_of_channels[i] = 1
    proxy.set_state_of_channels(state_of_channels)

In [5]:
# Get the current state of the channels
proxy.state_of_channels

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1], dtype=uint8)

In [6]:
# Set the voltage
proxy.voltage = 100

In [7]:
# Get the current voltage
proxy.voltage

100.0